In [1]:
import csv
import pandas as pd
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [14]:
def read_reports_from_csv():
    df = pd.read_csv("All_Scraped.csv")
    report_list = df["Report"].tolist()
    return report_list

previous_reports = read_reports_from_csv()

In [3]:
options = Options()
# options.add_argument('--headless=new')
# options.add_argument('--ignore-certificate-errors')
driver = webdriver.Chrome(options=options)
driver.maximize_window()

url = 'https://app.questn.com/communities'

driver.get(url)
element = WebDriverWait(driver, 15).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'div.style_communityBox__kzFrj')))

In [6]:
def get_data(projects, new_scraped_writer, all_scraped_writer):
    for project in projects:
        data = {}

        data['Report'] = project.get_attribute('href')

        if data['Report'] not in previous_reports:
            try:
                data['Name'] = project.find_element(By.CSS_SELECTOR, 'p.WalsheimBold').text
            except:
                continue

            data['Website'] = ''
            
            data['Telegram'] = ''

            data['Twitter'] = ''

            data['Discord'] = ''

            new_scraped_writer.writerow(data)
            all_scraped_writer.writerow(data)
        else:
            continue

In [ ]:
def scroll_to_end():
    last_project = driver.find_elements(By.CSS_SELECTOR, 'div.style_communityBox__kzFrj > a')[-1]
    driver.execute_script("arguments[0].scrollIntoView(true);", last_project)

    sleep(8)

In [15]:
header = ['Name', 'Report', 'Website','Telegram','Twitter','Discord']
with open("All_Scraped.csv", mode='a', newline='', encoding='utf-8') as all_scraped_file, open("New_Scraped.csv", mode='a', newline='', encoding='utf-8') as new_scraped_file:
    new_scraped_writer = csv.DictWriter(new_scraped_file, fieldnames=header)
    all_scraped_writer = csv.DictWriter(all_scraped_file, fieldnames=header)

    while True:
        old_page = driver.page_source
        scroll_to_end()
        new_page = driver.page_source

        # If there is no change in the page source, we have reached the end of the content
        if old_page == new_page:
            break
    
    projects = driver.find_elements(By.CSS_SELECTOR, 'div.style_communityBox__kzFrj > a')
    get_data(projects, new_scraped_writer, all_scraped_writer)